### Ce notebook est seulement pour creer une fonction qui nous permettra de generer des valeurs retardees pour les variables que l'on utilisera par la suite pour preduire quelle equipe qui devrait gagner...

In [ ]:
# Check Python Version
from platform import python_version
import pandas as pd
import requests
import datetime

pd.set_option('display.max_columns', None)

print(python_version())

In [34]:
df = pd.read_csv('../../data/games.csv')

In [35]:
df.head(5)

,Unnamed: 0,content.link,date,gameDate,gamePk,gameType,link,season,status.abstractGameState,status.codedGameState,status.detailedState,status.startTimeTBD,status.statusCode,teams.away.leagueRecord.losses,teams.away.leagueRecord.ot,teams.away.leagueRecord.type,teams.away.leagueRecord.wins,teams.away.score,teams.away.team.id,teams.away.team.link,teams.away.team.name,teams.home.leagueRecord.losses,teams.home.leagueRecord.ot,teams.home.leagueRecord.type,teams.home.leagueRecord.wins,teams.home.score,teams.home.team.id,teams.home.team.link,teams.home.team.name,venue.id,venue.link,venue.name
0,0,/api/v1/game/2017030115/content,2018-04-21,2018-04-21T19:00:00Z,2017030115,P,/api/v1/game/2017030115/feed/live,20172018,Final,7,Final,False,7,4,NaN,league,1,1,1,/api/v1/teams/1,New Jersey Devils,1,NaN,league,4,3,14,/api/v1/teams/14,Tampa Bay Lightning,5017.0,/api/v1/venues/5017,Amalie Arena
1,1,/api/v1/game/2017030135/content,2018-04-21,2018-04-21T19:00:00Z,2017030135,P,/api/v1/game/2017030135/feed/live,20172018,Final,7,Final,False,7,3,NaN,league,2,3,29,/api/v1/teams/29,Columbus Blue Jackets,2,NaN,league,3,4,15,/api/v1/teams/15,Washington Capitals,NaN,/api/v1/venues/null,Verizon Center
2,2,/api/v1/game/2017030125/content,2018-04-21,2018-04-22T00:00:00Z,2017030125,P,/api/v1/game/2017030125/feed/live,20172018,Final,7,Final,False,7,3,NaN,league,2,4,10,/api/v1/teams/10,Toronto Maple Leafs,2,NaN,league,3,3,6,/api/v1/teams/6,Boston Bruins,5085.0,/api/v1/venues/5085,TD Garden
3,3,/api/v1/game/2017030146/content,2018-04-22,2018-04-22T19:00:00Z,2017030146,P,/api/v1/game/2017030146/feed/live,20172018,Final,7,Final,False,7,2,NaN,league,4,8,5,/api/v1/teams/5,Pittsburgh Penguins,4,NaN,league,2,5,4,/api/v1/teams/4,Philadelphia Flyers,5096.0,/api/v1/venues/5096,Wells Fargo Center
4,4,/api/v1/game/2017030156/content,2018-04-22,2018-04-22T23:00:00Z,2017030156,P,/api/v1/game/2017030156/feed/live,20172018,Final,7,Final,False,7,2,NaN,league,4,5,18,/api/v1/teams/18,Nashville Predators,4,NaN,league,2,0,21,/api/v1/teams/21,Colorado Avalanche,5064.0,/api/v1/venues/5064,Pepsi Center


##### L'objectif est de faire une fonction que va creer des valeurs retardes (p) pour un ensemble de variables. Pour etre en mesure d'effectuer ces valeurs retardees, il faut d'abord faire un sort sur un ensemble de dimension pertinentes. Ces dimensions sont les suivantes:
+ Equipe 
+ Saison 
+ Date

In [22]:
df_sorted = df.sort_values(by=['teams.home.team.name', 'season' ,'gamePk'])

In [23]:
df_sorted['teams.away.score.L1'] = df_sorted.groupby(['teams.home.team.name', 'season'])['teams.away.score'].shift(1)

In [29]:
df_out = df_sorted[df_sorted['teams.home.team.name']=='Washington Capitals']

In [27]:
df_sorted['teams.home.team.name'].value_counts()

Vegas Golden Knights     56
Washington Capitals      56
Tampa Bay Lightning      55
Winnipeg Jets            54
Boston Bruins            53
San Jose Sharks          51
Nashville Predators      50
Toronto Maple Leafs      49
Calgary Flames           49
Pittsburgh Penguins      49
New York Islanders       48
Dallas Stars             48
Columbus Blue Jackets    48
Carolina Hurricanes      47
Colorado Avalanche       47
St. Louis Blues          47
Montréal Canadiens       46
Philadelphia Flyers      46
Los Angeles Kings        45
Edmonton Oilers          45
Vancouver Canucks        45
Minnesota Wild           45
Detroit Red Wings        45
Anaheim Ducks            44
Florida Panthers         44
Ottawa Senators          44
Chicago Blackhawks       44
Buffalo Sabres           44
New York Rangers         44
Arizona Coyotes          43
New Jersey Devils        42
Team Pacific              1
Team Atlantic             1
Team Central              1
Koln Haie                 1
Bern Sc Bern        

In [40]:
def create_lag(df, p, sortList, partList, lagList):
    """
    Cette fonction prend comme intrant un df, une liste de variables pour lesquelles il faut
    creer des valeurs retardees, le nombre de retard, les variables sur lesquelles il faut sort,
    les variables de partition et la liste de varibles pour lesquelles on veut des valeurs retardees.
    La fonction retourne un df augmente des variables retardees
    """
    df = df.sort_values(by=sortList)
    
    for feature in lagList:
        for i in range(p):
            df['{}.L{}'.format(feature,i+1)] = df.groupby(partList)[feature].shift(i+1)
    
    return df

In [41]:
lagged = create_lag(df=df,
                    p=2,
                    sortList=['teams.home.team.name', 'season', 'gamePk'], 
                    partList=['teams.home.team.name', 'season'],
                    lagList=['teams.home.score','teams.away.score'])

In [43]:
lagged.head(5)

,Unnamed: 0,content.link,date,gameDate,gamePk,gameType,link,season,status.abstractGameState,status.codedGameState,status.detailedState,status.startTimeTBD,status.statusCode,teams.away.leagueRecord.losses,teams.away.leagueRecord.ot,teams.away.leagueRecord.type,teams.away.leagueRecord.wins,teams.away.score,teams.away.team.id,teams.away.team.link,teams.away.team.name,teams.home.leagueRecord.losses,teams.home.leagueRecord.ot,teams.home.leagueRecord.type,teams.home.leagueRecord.wins,teams.home.score,teams.home.team.id,teams.home.team.link,teams.home.team.name,venue.id,venue.link,venue.name,teams.home.score.L1,teams.home.score.L2,teams.away.score.L1,teams.away.score.L2
86,86,/api/v1/game/2018010038/content,2018-09-20,2018-09-21T02:00:00Z,2018010038,PR,/api/v1/game/2018010038/feed/live,20182019,Final,6,Final,False,6,0,0.0,league,2,7,28,/api/v1/teams/28,San Jose Sharks,2,0.0,league,0,3,24,/api/v1/teams/24,Anaheim Ducks,5046.0,/api/v1/venues/5046,Honda Center,NaN,NaN,NaN,NaN
116,116,/api/v1/game/2018010068/content,2018-09-24,2018-09-25T02:00:00Z,2018010068,PR,/api/v1/game/2018010068/feed/live,20182019,Final,6,Final,False,6,2,0.0,league,3,2,53,/api/v1/teams/53,Arizona Coyotes,3,0.0,league,1,4,24,/api/v1/teams/24,Anaheim Ducks,5046.0,/api/v1/venues/5046,Honda Center,3.0,NaN,7.0,NaN
131,131,/api/v1/game/2018010083/content,2018-09-26,2018-09-27T02:00:00Z,2018010083,PR,/api/v1/game/2018010083/feed/live,20182019,Final,6,Final,False,6,4,1.0,league,1,4,26,/api/v1/teams/26,Los Angeles Kings,3,0.0,league,2,7,24,/api/v1/teams/24,Anaheim Ducks,5046.0,/api/v1/venues/5046,Honda Center,4.0,3.0,2.0,7.0
193,193,/api/v1/game/2018020035/content,2018-10-08,2018-10-09T02:00:00Z,2018020035,R,/api/v1/game/2018020035/feed/live,20182019,Final,7,Final,False,7,1,2.0,league,0,2,17,/api/v1/teams/17,Detroit Red Wings,0,0.0,league,3,3,24,/api/v1/teams/24,Anaheim Ducks,5046.0,/api/v1/venues/5046,Honda Center,7.0,4.0,4.0,2.0
202,202,/api/v1/game/2018020044/content,2018-10-10,2018-10-11T02:00:00Z,2018020044,R,/api/v1/game/2018020044/feed/live,20182019,Final,7,Final,False,7,2,0.0,league,1,3,53,/api/v1/teams/53,Arizona Coyotes,0,1.0,league,3,2,24,/api/v1/teams/24,Anaheim Ducks,5046.0,/api/v1/venues/5046,Honda Center,3.0,7.0,2.0,4.0
